In [179]:
#!/usr/bin/python
# -*- coding:utf-8 -*-
#standard python packages
import numpy as np
import math
import matplotlib.pyplot as plt

##################################################
##changeable things, for now only the first part of getting the total maximum will work if several files are wanted
##for a single file all cells work

#outname = "text_files/" +  "TLE_max_2017-06-28.txt"    #in case for saving
#namelist = ["BlueJet1_128GTU.txt", "BlueJet2_128GTU.txt", "BlueJet3_128GTU.txt", "BlueJet5_128GTU.txt", "BlueJet6_128GTU.txt", "BlueJet7_128GTU.txt", "Elves1_128GTU.txt", "Elves2_128GTU.txt", "Sprite1_128GTU.txt", "Sprite2_128GTU.txt", "Meteor_1_L2.txt", "Meteor_2_L2.txt"]    #in case of wanting to go through multiple files
#saving = 1    #0 = saves to file, anything else will only run

##below is for testing, create outname file before saving or the saving part needs to be done twice since the first time only an empty file is created
outname = "text_files/" +  "test_max.txt"
#namelist = ["test_composite.txt"]
namelist = ["Sprite1_128GTU.txt"]
saving = 0    #0 = saves to file, anything else will only run
show = 0    #0 = shows plot, anything else will only run

##################################################

maximum_collection = []
total_ind = 0
for name in namelist:
    total_list = []    #list of the maxmimum number for each timestep in the file
    tempname = "text_files/" + name
    with open(tempname) as f:
        data = []
        for line in f:
            line = line.split()
            if line:
                line = [int(i) for i in line]
                data.append(line)    #making the text file into a list

        length = np.shape(data)[0]    #number of rows in the text file
        ind = range(0, length, 49)    #list of numbers with the indices in the list data indicating change in timestep, [0, 49, 98, 147....]
        total_ind = total_ind + len(ind)
        nz = int(len(ind))    #nz is the total number of timesteps in the current textfile
        counts = np.zeros((nz, 2304))    #list of nz lists each with 2304 zeros corresponding to each pixel
        for iz in range(nz):
            counts[iz][:] = np.concatenate((data[ind[iz] + 1 : ind[iz] + 49]), axis = 0)    #filling the nz lists with the data

    for maximum in counts:
        total_list.append(max(maximum))

    maximum_collection.append(name + "  " + str(max(total_list)))


In [180]:
#for saving the maximum for each text file opened
if saving == 0:
    thefile = open(outname, 'w')
    for line in maximum_collection:    #goes through the total_list and writes each line in the text file
        thefile.write(line + "\n")

In [181]:
##below is to find the pixel with the maximum, does not work for multiple files at the same time since total_list gets overwritten
## in case multiple files is wanted, run make_composite_texts first on the wanted files and then this on the composite text
pixel = []
x_max = 0
y_max = 0
for a in range(nz):
    w = counts[a].reshape(48, 48)    #this makes the array of 2304 numbers in a 48x48 shape
    temp_y,temp_x = np.where(w==total_list[a])    #this gives the pixels which are maximum
    y_max_temp,x_max_temp = np.where(w==max(total_list))    #this gives the pixels which are maximum
    if len(y_max_temp) > 0:
        y_max = y_max_temp
        x_max = x_max_temp
    for q in range(len(temp_x)):    #loop to make the pixel into a format that is easier to read
        temp = str(a+1) + "; (" + str(temp_x[q]+1) + "," + str(48-temp_y[q]) + "); " + str(total_list[a])    #to make it look good
        pixel.append(temp)    #saves in the following format: "frame#; (x,y); maximum"

##below is for printing the pixel
#for a in range(len(pixel)):
#    print pixel[a]


In [182]:
#for saving the maximum pixel for each frame (will be very long if the files are unintegrated)
if saving == 0:
    thefile = open("text_files/pixel_max.txt", 'w')
    for line in pixel:    #goes through the total_list and writes each line in the text file
        thefile.write(line + "\n")


In [183]:
#below is for making a plot over how the maximum pixel changes over all timestamps
indexnumber = 48*y_max[0] + x_max[0]
#print "Index number in counts for the maximum pixel is " + str(indexnumber)
max_pixel_change = []
plot_savename = "plots/" + namelist[0].strip(".txt")    #this will be the name the plot is saved as, no file ending
for a in counts:
    max_pixel_change.append(a[indexnumber])
x_range = range(1,len(max_pixel_change)+1)
fig = plt.figure()
ax = plt.gca()
plt.plot(x_range, max_pixel_change)
plt.title('Filename = ' + namelist[0])
plt.ylabel('counts? kR?')
plt.xlabel('Timestamp')
if show == 0:
    plt.show()

In [184]:
#for saving how the maximum pixel changes over each timestamp
if saving == 0:
    fig.savefig(plot_savename + "_max.eps")
